# Test SSO Login

The instance has all the permissions required, now run the login command for the `Jupyter-IR-ViewOnly` role and the loggin account.

In [3]:
import boto3, os
from jupyterirtools import sso

sso.login("Jupyter-IR-ViewOnly")

Credentials expire in 5 hours and 46 minutes


In [4]:
sso.print_permissions()

Account: 251344881676: Role: Jupyter-IR-AdministratorAccess
Account: 251344881676: Role: Jupyter-IR-ViewOnly
Account: 913149361159: Role: Jupyter-IR-AdministratorAccess
Account: 959309154643: Role: Jupyter-IR-AdministratorAccess
Account: 383086473915: Role: AdministratorAccess
Account: 383086473915: Role: Jupyter-IR-AdministratorAccess
Account: 383086473915: Role: Jupyter-IR-SysAdminAccess
Account: 383086473915: Role: Jupyter-IR-ViewOnly
Account: 669399753706: Role: Jupyter-IR-AdministratorAccess


In [5]:
boto3.setup_default_session()
sts = boto3.client('sts')
identity = sts.get_caller_identity()
print(f"UserId: {identity['UserId']}\nAccount: {identity['Account']}\n")

UserId: AROATVBKSRAGLFBLNEXHW:sso_user
Account: 251344881676



In [6]:
session = sso.get_session("Jupyter-IR-AdministratorAccess",os.environ['MANAGEMENT_ACCOUNT'])
sts = session.client('sts')
identity = sts.get_caller_identity()
print(f"UserId: {identity['UserId']}\nAccount: {identity['Account']}\n")

UserId: AROAVSMN3WK57E362JKYS:sso_user
Account: 383086473915



In [ ]:
! aws sts get-caller-identity


In [ ]:
sso.logout()

In [ ]:
boto3.setup_default_session()
sts = boto3.client('sts')
try:
    sts.get_caller_identity()
    print("The logout FAILED")
except:
    print("The logout was successful")